# 集成学习：XGBoost
- GBDT在模型训练时只使用了代价函数的一阶导数信息，XGBoost对代价函数进行二阶泰勒展开，可以同时使用一阶和二阶导数。

# 一 XGBoost的基础思想与实现

## 1 XGBoost pk 梯度提升树

极限提升树XGBoost(Extreme Gradient Boosting，XGB，发音/æks-g-boost/)是基于梯度提升树GBDT全面升级的新一代提升算法，也是提升家族中最富盛名、最灵活、最被机器学习竞赛所青睐的算法。不同于我们之前学过的任意单一算法，XGBoost是一个以提升树为核心的算法系统，它覆盖了至少3+建树流程、10+损失函数，可以实现各种类型的梯度提升树，灵活性无与伦比。同时，XGBoost天生被设计成支持巨量数据，因此可以自由接入GPU/分布式/数据库等系统、还创新了众多工程上对传统提升算法进行加速的新方法。可以说，XGBoost是21世纪中Boosting算法的又一个里程碑，它开创了后GBDT时代中Boosting算法的新辉煌。

**<font color="red">需要注意的是，学习XGBoost需要大量梯度提升树（GBDT）相关知识，本课程将假设你已经非常熟悉梯度提升树的原理与特点、并且熟悉sklearn中的交叉验证的用法。如果你不熟悉梯度提升树，强烈建议你回顾之前的课程。</font>**

作为Boosting算法，XGBoost中自然包含Boosting三要素：

- 损失函数$L(y,\hat{y})$ ：用以衡量模型预测结果与真实结果的差异。<br>
- 弱评估器$f(x)$ ：（一般为）决策树，不同的boosting算法使用不同的建树过程。<br>
- 综合集成结果$H(x)$：即集成算法具体如何输出集成结果

并且，XGBoost也遵循Boosting算法的基本流程进行建模：

---
**<font color="green"><center>依据上一个弱评估器$f(x)_{k-1}$的结果，计算损失函数$L$，
    <br>并使用$L$自适应地影响下一个弱评估器$f(x)_k$的构建。<br>集成模型输出的结果，受到整体所有弱评估器$f(x)_0$ ~ $f(x)_K$的影响。</center></font>**
---

当然，XGBoost真实的流程比上述过程复杂得多。虽然梯度提升树的流程本身已经比较复杂，XGBoost还是在此流程上做出了众多关键的改进。综合来看，这些改进都是基于XGBoost中两种非常关键的思想实现的：

- **第一，实现精确性与复杂度之间的平衡**<br>

树的集成模型是机器学习中最为强大的学习器之一，这一族学习器的特点是精确性好、适用于各种场景，但运行缓慢、且过拟合风险很高，因此从学习单一决策树时起，我们就持续为大家提供丰富的剪枝策略，目的就是为了降低各种树模型的模型复杂度，从而控制住过拟合。**树模型的学习能力与过拟合风险之间的平衡，就是预测精确性与模型复杂度之间的平衡，也是经验风险与结构风险之间的平衡**，这一平衡对决策树以及树的集成模型来说是永恒的议题。<br>

在过去，我们总是先建立效果优异的模型，再依赖于手动剪枝来调节树模型的复杂度，但在XGBoost中，精确性与复杂度会在训练的每一步被考虑到。主要体现在：

> - **1. XGBoost为损失函数$L(y,\hat{y})$加入结构风险项，构成目标函数$O(y,\hat{y})$**<br><br>
> 在AdaBoost与GBDT当中，我们的目标是找到损失函数$L(y,\hat{y})$的最小值，也就是让预测结果与真实结果差异最小，这一流程只关心精确性、不关心复杂度和过拟合情况。为应对这个问题，XGBoost从决策树的预剪枝流程、逻辑回归、岭回归、Lasso等经典算法的抗过拟合流程吸取经验，在损失函数中加入了控制过拟合的结构风险项，并将【$L(y,\hat{y})$ + 结构风险】定义为目标函数$O(y,\hat{y})$。<br><br>
> 这一变化让XGBoost在许多方面都与其他Boosting算法不同：例如，XGBoost是向着令目标函数最小化的目标进行训练，而不是令损失函数最小化的方向。再比如，XGBoost会优先利用结构风险中的参数来控制过拟合，而不像其他树的集成模型一样依赖于树结构参数（例如`max_depth`，`min_impurity_decrease`等）。<br><br>
> - **2. 使用全新不纯度衡量指标，将复杂度纳入分枝规则**<br><br>
> 在之前学过的算法当中，无论Boosting流程如何进化，建立单棵决策树的规则基本都遵循我们曾经学过的CART树流程，在分类树中，我们使用信息增益（information gain）来衡量叶子的质量，在回归树中，我们使用MSE或者弗里德曼MSE来衡量叶子的质量。这一流程有成熟的剪枝机制、预测精度高、能够适应各种场景，但却可能建立复杂度很高的树。<br><br>
> 为实现精确性与复杂度之间的平衡，XGBoost重新设定了分枝指标**【结构分数】**（原论文中写作Structure Score，也被称为质量分数Quality Score），以及基于结构分数的**【结构分数增益】**（Gain of structure score），结构分数增益可以逼迫决策树向整体结构更简单的方向生长。<br><br>
> 这一变化让XGBoost使用与传统CART略有区别的建树流程，同时在建树过程中大量使用残差（Residuals）或类残差对象作为中间变量，因此XGBoost的数学过程比其他Boosting算法更复杂。

- **第二，极大程度地降低模型复杂度、提升模型运行效率，将算法武装成更加适合于大数据的算法**<br>

在任意决策树的建树过程中，都需要对每一个特征上所有潜在的分枝节点进行不纯度计算，当数据量巨大时，这一计算将消耗巨量的时间，因此树集成模型的关键缺点之一就是计算缓慢，而这一缺点在实际工业环境当中是相当致命的。为了提升树模型的运算速度、同时又不极大地伤害模型的精确性，XGBoost使用多种优化技巧来实现效率提升：<br>

> - **1. 使用估计贪婪算法、平行学习、分位数草图算法等方法构建了适用于大数据的全新建树流程**<br><br>
> - **2. 使用感知缓存访问技术与核外计算技术，提升算法在硬件上的运算性能**<br><br>
> - **3. 引入Dropout技术，为整体建树流程增加更多随机性、让算法适应更大数据**<br><br>
> 不仅在数学方法上有所改进，XGBoost正式拉开了Boosting算法工程优化的序幕。后续更多的Boosting算法，包括LightGBM，CatBoost等也都是在工程方法上做出了大量的优化。遗憾的是，XGBoost的平行学习、估计贪婪算法等知识点将不会被包括在本次课程当中。在讲解LightGBM的时候，我们将详解基于直方图的估计算法，那时我们就能够很容易地理解XGBoost所使用的估计贪婪算法了。

除此之外，XGBoost还保留了部分与梯度提升树类似的属性，包括：

- **弱评估器的输出类型与集成算法输出类型不一致**

> 对于AdaBoost或随机森林算法来说，当集成算法执行的是回归任务时，弱评估器也是回归器，当集成算法执行分类任务时，弱评估器也是分类器。但对于GBDT以及基于GBDT的复杂Boosting算法们而言，**无论集成算法整体在执行回归/分类/排序任务，弱评估器一定是回归器**。GBDT通过sigmoid或softmax函数输出具体的分类结果，但实际弱评估器一定是回归器，XGBoost也是如此。

- **拟合负梯度，且当损失函数是0.5倍MSE时，拟合残差**

> 任意Boosting算法都有自适应调整弱评估器的步骤。在GBDT当中，每次用于建立弱评估器的是样本$X$以及当下集成输出$H(x_i)$与真实标签$y$之间的伪残差（也就是负梯度）。当损失函数是$\frac{1}{2}MSE$时，负梯度在数学上等同于残差（Residual），因此**GBDT是通过拟合残差来影响后续弱评估器结构**。XGBoost也是依赖于拟合残差来影响后续弱评估器结构，但是与GBDT一样，这一点需要通过数学来证明。

- **抽样思想**

> GBDT借鉴了大量Bagging算法中的抽样思想，XGBoost也继承了这一属性，因此在XGBoost当中，我们也可以对样本和特征进行抽样来增大弱评估器之间的独立性

因为存在这些相似之处，因此我们将会在XGBoost的参数中看到部分熟悉的参数，如果你对梯度提升树足够熟悉，那XGBoost的许多参数对你来说应该并不难懂。需要注意的是，作为2014年才被正式提出的Boosting算法，XGBoost是一个独立于经典算法的算法系统，因此xgboost库是需要单独安装的。

你可以使用以下代码在cmd中非常简单地安装xgboost，通常来说无需配置环境变量。

In [ ]:
#!pip install xgboost #安装xgboost库
#!pip install --upgrade xgboost #更新xgboost库

In [ ]:
import xgboost as xgb #导入成功则说明安装正确

In [ ]:
xgb.__version__

'1.5.1'

#  原理进阶：XGBoost的求解流程

**<font color="red">学习XGBoost的数学原理需要大量梯度提升树（GBDT）相关知识，本课程将假设你已经非常熟悉梯度提升树的原理与特点。如果你不熟悉梯度提升树，强烈建议你回顾之前的课程。</font>**

## 1 XGBoost的基本数学流程

作为Boosting算法的里程碑，XGBoost以它高度复杂的数学原理闻名。无论你是阅读XGBoost原始论文，还是寻找经典书籍作为资料，都会发现数学占了很大的篇幅。然而事实上，随着Boosting算法的数学过程越来越复杂，实际建模过程和实现代码却是越来越简单的。**数学并没有让算法变得更复杂，而是让算法变得更简单**。

- **参考文献**

XGBoost最初是由陈天奇及华盛顿大学团队在2014年提出，在提出时就已经具有了今天我们要证明的全套复杂数学理论。但在搭建xgboost算法库的过程中，研发团队不断吸收当代各种经典算法的元素，形成更加复杂的算法系统：例如，DART树是吸收了深度学习中的Dropout技术，我们没有详细讲解的基于直方图的估计贪婪算法是借鉴LightGBM的技术，参考以下论文：

《XGBoost：一种可拓展的提升树系统》
Chen,T.Q.; Geustrin, C. (2014). "XGBoost: A Scalable Tree Boosting System"

《DART树：当Dropout遇见自适应回归树》
Rashmi, K.V.; Ran Gilad-Bachrach.(2015) “DART: Dropouts meet Multiple Additive Regression Trees”

《Lightgbm：一种极度高效的梯度提升树》
Ke, G. et al.(2017)."Lightgbm: A highly efficient gradient boosting decision tree"

你可以在本周上传的课程中找到以上三篇论文。需要注意的是，在本节中我们将会专注于最原始的XGBoost论文，而不会涉及到其他两篇文献的内容。其中，XGBoost基本数学流程是总结自GBDT的流程与XGBoost论文中涉及到的伪代码，而所有证明过程都来自于XGBoost原始论文。同时，为了与之前的课程保持一致，我们将使用与原论文略有不同的符号，但数学过程将是完全相同的。在阅读原论文时，记得辨析不同的数学符号。

- **基本流程**

不要忘记，对任何Boosting算法来说我们都有如下流程（对大部分算法来说是损失函数$L$，对XGBoost来说是目标函数$O$）：

---
**<font color="green"><center>依据上一个弱评估器$f(x)_{k-1}$的结果，计算目标函数$O$，
    <br>并使用$O$自适应地影响下一个弱评估器$f(x)_k$的构建。<br>集成模型输出的结果，受到整体所有弱评估器$f(x)_0$ ~ $f(x)_K$的影响。</center></font>**
---

假设现有数据集$N$，含有形如$(x_i,y_i)$的样本$M$个，$i$为任意样本的编号，单一样本的损失函数为$l(y_i,H(x_i))$，其中$H(x_i)$是$i$号样本在集成算法上的预测结果，整个算法的损失函数为$L(y,H(x))$，且总损失等于全部样本的损失之和：$L(y,H(x)) = \sum_i l(y_i,H(x_i))$。目标函数中使用L2正则化（$\lambda$为0，$\alpha$为0），并且$\gamma$不为0。

同时，弱评估器为回归树$f$，总共学习$K$轮（注意在GBDT当中我们使用的是大写字母T来表示迭代次数，由于在XGBoost当中字母T被用于表示目标函数中的叶子总量，因此我们在这里使用字母K表示迭代次数）。则XGBoost回归的基本流程如下所示：

- **1) 初始化**

初始化数据迭代的起点$H_0(x)$。在应用xgboost时，我们可以指定任意数字来作为$H_0(x)$，但在xgboost原始论文当中，并未详细讨论如何计算迭代的初始值。考虑到XGBoost在许多方面继承了梯度提升树GBDT的思想，我们可以使用公式来计算XGBoost的$H_0$：

$$
\begin{aligned}
H_0(x) &= \mathop{argmin}_{C} \sum_{i=1}^M l(y_i,C)\\ \\
&= \mathop{argmin}_{C} L(y,C)
\end{aligned}
$$

其中$y_i$为真实标签，$C$为任意常数。以上式子表示，找出令$\sum_{i=1}^Ml(y_i,C)$最小的常数$C$值，并输出最小的$\sum_{i=1}^Ml(y_i,C)$作为$H_0(x)$的值。需要注意的是，由于$H_0(x)$是由全部样本的$l$计算出来的，因此所有样本的初始值都是$H_0(x)$，不存在针对某一样本的单一初始值。

由于在初始的时候没有树结构，因此没有复杂度等信息，因此没有使用目标函数求初始值，而是使用了损失函数。在GBDT的数学过程当中，我们详细展示过如何求解令初始损失最小的$C$（对损失求一阶导数并让一阶导数为0），并且我们详细证明过，当损失函数为MSE时，令整体初始损失最小的$C$值就是$y$的均值。对XGBoost来说这一切都成立，只不过在xgboost库中我们默认的初始值为0.5。
<br>

开始循环，for k in 1,2,3...K:

> - **2) 抽样**<br><br>
在现有数据集$N$中，抽样$M$ * `subsample`个样本，构成训练集$N^k$<br><br>
> - **3) 求拟合项**<br><br>
对任意一个样本$i$，计算一阶导数$g_{ik}$，二阶导数$h_{ik}$，以及伪残差（pseudo-residuals）$r_{ik}$，具体公式为：<br><br>
> $$g_{ik} = \frac{\partial{l(y_i,H_{k-1}(x_i))}}{\partial{H_{k-1}(x_i)}}
$$<br>
> $$h_{ik} = \frac{\partial^2{l(y_i,H_{k-1}(x_i))}}{\partial{H^2_{k-1}(x_i)}}
$$<br>
> $$r_{ik} = -\frac{g_{ik}}{h_{ik}}$$

> 不难发现，伪残差是一个样本的一阶导数除以二阶导数并取负的结果，并且在进行第k次迭代、计算第k个导数时，我们使用的是前k-1次迭代后输出的集成算法结果。同时，我们是先对目标函数$l$中的自变量$H(x)$求导，再令求导后的结果等于$H_{t-1}(x_i)$的值，并不是直接对$H_{t-1}(x_i)$这一常数求导。<br><br>
> 对常数求导，以及对变量求导是两个概念，举例说明：<br><br>
> $$l = x^2+x$$
> 对常数0求导：$$\frac{\partial{l}}{\partial{0}} = \frac{\partial{(x^2 + x)}}{0} = 0$$<br>
> 对变量x求导并让x=0：$$\frac{\partial{l}}{\partial{x}} = \frac{\partial{(x^2 + x)}}{\partial x} = 2x + 1 = 2*0 + 1 = 1$$<br>
> 因此，$g_{ik}$标准的写法应该是：
> $$g_{ik} = \big[\frac{\partial{l(y_i,H(x_i))}}{\partial{H(x_i)}}\big]_{H(x_i) = H_{k-1}\ \ (x_i)}
>$$<br>
> 在实际推导过程中，为公式简洁，简写为上述流程中的写法。<br><br>
> 在k=1时，所有求导计算过程中的$H_{k-1}(x_i)$都等于初始$H_0(x)$，在k>1时，每个样本上的$H_{k-1}(x_i)$都是不同的取值。<br>

> - **4) 建树**<br><br>
求解出伪残差后，在数据集$(x_i, r_{ik})$上按`colsample_by*`规则进行抽样，再按照**结构分数增益**规则建立一棵回归树$f_k$。注意在这个过程中，训练时拟合的标签为样本的伪残差$r_{ik}$，并且叶子节点$j$的结构分数和任意分枝时的结构分数增益的公式为：<br><br>
>$$Score_j = \frac{(\sum_{i \in j}g_i)^2}{\sum_{i \in j}h_i + \lambda}
>$$<br>
>$$Gain = \frac{1}{2} \left( \frac{(\sum_{i \in L}g_i)^2}{\sum_{i \in L}h_i + \lambda} + \frac{(\sum_{i \in R}g_i)^2}{\sum_{i \in R}h_i + \lambda} - \frac{(\sum_{i \in P}g_i)^2}{\sum_{i \in P}h_i + \lambda} \right) - \gamma
>$$ <br>
>建树过程不影响任何$g_{ik}$与$h_{ik}$的值。<br><br>
> - **5) 输出树上的结果**<br><br>
建树之后，依据回归树$f_k$的结构输出叶子节点上的输出值（预测值）。对任意叶子节点$j$来说，输出值为：<br><br>
> $$w_j = -\frac{\sum_{i \in j}g_{ik}}{\sum_{i \in j}h_{ik} + \lambda}
>$$<br>
> 假设样本$i$被分割到叶子$j$上，则有：<br><br>
> $$f_k(x_i) = w_j$$

> 使用字母$w$表示叶子节点上的输出值是XGBoost论文所规定的，我们曾经见过一次$w$，你还记得在哪里吗？在我们介绍XGBoost的目标函数时，L2正则项的表达式为$\frac{1}{2} \lambda \sum_{j=1}^T w_j^2$。我们曾说过$w$代表XGBoost中的叶子权重，**实际上叶子权重就是叶子上的输出值**。为不和其他权重混淆，之后我们统一称呼$w$为输出值。<br><br>
>不难发现，叶子节点上的输出值与结构分数很相似，只不过结构分数的分子上是平方，而输出值的分子上没有平方。在数学上我们可以证明，**该输出值能让目标函数最快减小**。<br><br>
> 你可能注意到了，在迭代刚开始时我们已经知道了输出值式子中所需的所有$g$和$h$。为什么还要建树呢？只有当我们建立了决策树，我们才能够知道具体哪些样本$i$在叶子节点$j$上。因此树$f_k$提供的是结构信息。<br><br>
> 由于任意样本必然被分到任意叶子上，因此对整棵树$f_k$来说，任意$f_k(x_i)$一定有对应的$w$。<br>

> - **6) 迭代**<br><br>
根据预测结果$f_k(x_i)$迭代模型，具体来说：<br><br>
> $$H_k(x_i) = H_{k-1}(x_i) + f_k(x_i)
>$$<br>
> 假设输入的步长为$\eta$，则$H_k(x)$应该为：<br><br>
> $$H_k(x_i) = H_{k-1}(x_i) + \eta f_k(x_i)
>$$<br>
> 对整个算法则有：<br><br>
> $$H_k(x) = H_{k-1}(x) + \eta f_k(x)
>$$<br>

- **7) 循环结束**<br><br>输出$H_K(x)$的值作为集成模型的输出值。

以上就是XGBoost的完整数学流程。不难发现，作为从GBDT改进而来的算法，XGBoost在基础数学流程上基本继承了GBDT的流程（7步走的流程与GBDT一模一样，同时也有继承伪残差等细节），但又在具体每个流程中都做出了改进，进一步简化了Boosting算法的运算流程——比如说，虽然整个算法持续再向降低目标函数的方向运行，但整个过程中不存在任何的**求最优解**的数学计算。除了建树流程以外，其他流程都是非常简单的按公式计算而已。

由于XGBoost原始论文中并不存在上述流程的完整说明，因此要梳理出该流程并不容易，但如果我们对GBDT的流程相对熟悉，那XGBoost的流程也并不难。对XGBoost来说，真正难度较大的部分并不是梳理以上算法流程，而是证明这一流程可以让模型向着目标函数最小化的方向运行。在这个流程中包括如下很明显的问题：

- 1. 建树时拟合的$r_{ik} = -\frac{g_{ik}}{h_{ik}}$究竟是什么？拟合它有什么意义？

- 2. 结构分数和结构分数增益的公式是如何推导出来的？为什么这样建树可以提升模型的效果？

- 3. 为什么叶子节点的输出值$w_j$是$-\frac{(\sum_{i \in j} g_{ik})}{\sum_{i \in j} h_{ik} + \lambda}$？这样输出有什么意义？

- 4. 课程的第一部分说XGBoost拟合的也是残差，残差在哪里？

接下来我们就将展示完整的证明来回答这些问题。

## 2 化简XGBoost的目标函数

- **定义目标函数与目标函数的自变量**

首先，根据之前对目标函数的定义，**XGBoost中目标函数是针对一棵树的目标函数**，而不是针对一个样本或一整个算法的目标函数。并且，任意树的目标函数都包括三大部分：损失函数$l$、叶子数量$T$以及正则项。具体地来说：

假设单一树$f_k$的目标函数为$O_k$，总共有$T$片叶子，该树上任意样本$i$的损失函数为$l((y_i,H(x_i))$，其中$H(x_i)$是$i$号样本在集成算法上的预测结果。树上总共有M个样本，目标函数中使用L2正则化（$\lambda$不为0，$\alpha$为0），并且$\gamma$不为0，则该树的目标函数为：

$$O_k = \sum_{i=1}^Ml(y_i,H_k(x_i)) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^Tw_j^2$$

**我们的目标是令目标函数最小，并找出令目标函数最小的某个自变量**。对使用普通损失函数的Boosting算法来说，算法的输出值$H(x)$是在迭代过程中不断变化的，损失函数$l(y,H(x))$也是在迭代中不断变小的：

$$H_k(x_i) = H_{k-1}(x_i) + f_k(x_i)$$

$$l_k = l(y_i,H_{k-1}(x_i) + f_k(x_i))$$

当迭代到第$k$次时，损失函数中的$y_i$与$H_{k-1}(x_i)$都是常数，只有$f_k(x_i)$是变量，因此我们只需要在损失函数上对$f_k(x_i)$求导，并找到令整体损失函数最小的预测值$f_k(x_i)$即可。在GBDT当中，我们提到过，**无论弱评估器$f_k$是什么结构、什么规则、如何建立、如何拟合，只要其最终的输出值$f_k(x_i)$是令整体损失函数$L$最小化的$f_k(x_i)$，那随着算法逐步迭代，损失函数必然会越来越小**。因此，一个适合的$f_k(x_i)$不仅能保证损失持续减小，还可以指导单个评估器的建立。

在GBDT当中，我们证明了令GBDT整体损失函数最小化的$f_k(x_i)$就是损失函数的负梯度$-g_i$（详见GBDT课程《四 4：证明：拟合伪残差可以令损失函数最快地减小》），也因此GBDT在建树时拟合负梯度。当损失函数为$\frac{1}{2}MSE$时，GBDT中的负梯度在数值上就等于残差，因此GBDT是拟合残差的算法。

在XGBoost当中，我们也可以对目标函数求导、并找出令目标函数最小的某个自变量，但问题在于，XGBoost的目标函数中存在多个自变量：

$$\begin{aligned} O_k &= \sum_{i=1}^Ml(y_i,H_k(x_i)) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^Tw_j^2 \\ &= \sum_{i=1}^M l \left( y_i,H_{k-1}(x_i) + \boldsymbol{\color{red}{f_k(x_i)}} \right) + \gamma \boldsymbol{\color{red}T} + \frac{1}{2}\lambda\sum_{j=1}^T\boldsymbol{\color{red}{w_j}}^2 \end{aligned}$$

其中，$T$是第$k$棵树上的叶子总量，$f_k(x_i)$与$w_j$都是模型输出的预测值（叶子上的输出值），不过表现形式不同，对任意位于叶子$j$上的样本$i$而言，数值上$f_k(x_i) = w_j$。对XGBoost来说，只能选择一个变量作为自变量，**考虑到$f_k(x_i)$只与单个样本的精确程度有关，而$T$只与树结构有关，XGBoost论文最终选择了即与精确度有关、又与树结构有关的变量**$w_j$。同时，如果知道叶子的最佳输出值$w_j$就可以引导树成长为合理的结构，但只知道叶子的总量$T$是无法指导建树的。

因此，求解XGBoost目标函数的第一步，就是将目标函数尽量整理成以$w_j$表示的形式。

- **对目标函数进行泰勒展开**

在GBDT当中，我们借助一阶泰勒展开化简了损失函数，借鉴于GBDT的思路，XGBoost也使用了泰勒展开。具体来看：

在数学中，泰勒级数使用无限个项的连加式来表示一个函数。实际应用当中，我们一般取有限项的连加式来逼近一个函数。当总共有N项时，连加式被叫做N阶泰勒展开（Nth-order Taylor approximation）。假设现在存在函数$f(x)$，则有：

> - **泰勒级数（无限项）**： 
$$f(x) = \sum_{n=0}^{\infty}\frac{f^{(n)}(a)}{n!}(x-a)^n$$

其中(x-a)是非常小的任意实数/复数，$n!$是n的阶乘，$f^{(n)}(a)$是函数$f(x)$的n阶导数在a点的取值。当a为0时，泰勒级数也被叫做麦克劳思级数。

> - **一阶泰勒展开**：
$$\begin{aligned}
f(x) &\approx \sum_{n=0}^{1}\frac{f^{(n)}(a)}{n!}(x-a)^n \\
&\approx f(a) + \frac{f'(a)}{1!}(x-a)
\end{aligned}
$$

> - **二阶泰勒展开**：
$$\begin{aligned}
f(x) &\approx \sum_{n=0}^{2}\frac{f^{(n)}(a)}{n!}(x-a)^n \\
&\approx f(a) + \frac{f'(a)}{1!}(x-a) + \frac{f''(a)}{2!}(x-a)^2
\end{aligned}
$$

> - **N阶泰勒展开**：
$$\begin{aligned}
f(x) &\approx \sum_{n=0}^{N}\frac{f^{(n)}(a)}{n!}(x-a)^n \\
&\approx f(a) + \frac{f'(a)}{1!}(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f'''(a)}{3!}(x-a)^3 + ...
\end{aligned}
$$

阶数越大，泰勒展开的值越接近$f(x)$的真实值。

在我们的目标函数$O_k$中，**可以被泰勒展开的是第一部分损失函数**$l$：

$$O_k = \sum_{i=1}^Ml \left( y_i,H_{k-1}(x_i) + f_k(x_i) \right) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2$$

由于损失函数$l$中只有唯一变量$H_{k-1}(x_i) + f_k(x_i)$，因此可以将函数简写为$l(H_{k-1}(x_i) + f_k(x_i))$。

根据二阶泰勒展开，已知：

$$\begin{aligned}
f(x) &\approx \sum_{n=0}^{2}\frac{f^{(n)}(a)}{n!}(x-a)^n \\
&\approx f(a) + \frac{f'(a)}{1!}(x-a) + \frac{f''(a)}{2!}(x-a)^2
\end{aligned}
$$

令泰勒展开中的$x = H_{k-1}(x_i) + f_k(x_i)$，令泰勒展开中的$a = H_{k-1}(x_i)$，则$(x-a) = f_k(x_i)$。据此，损失函数$l(H_{k-1}(x_i) + f_k(x_i))$可以被表示为：

$$\begin{aligned}
l(H_{k-1}(x_i) + f_k(x_i)) &\approx l(H_{k-1}(x_i)) + \frac{\partial{l(H_{k-1}(x_i))}}{\partial{H_{k-1}(x_i)}} * f_k(x_i) +  \frac{\partial^2{l(H_{k-1}(x_i))}}{2\partial{H^2_{k-1}(x_i)}} * f^2_k(x_i)\\
\end{aligned}
$$

在XGBoost中我们定义过损失函数的一阶导数与二阶导数：

$$g_{ik} = \frac{\partial{l(y_i,H_{k-1}(x_i))}}{\partial{H_{t-1}(x_i)}}$$

$$h_{ik} = \frac{\partial^2{l(y_i,H_{k-1}(x_i))}}{\partial{H^2_{t-1}(x_i)}}$$

**在XGBoost原论文中，为了公式简洁，$g_i$和$h_i$并没有呈现下标$k$，但我们已经很清楚：$g$与$h$是在每一轮迭代时需要被重新计算的**。在这里我们也参照原论文中的做法去掉下标$k$。因此，经过泰勒展开后的式子可以化简为：

$$\begin{aligned}l(H_{k-1}(x_i) + f_k(x_i)) &\approx l(H_{k-1}(x_i)) + g_if_k(x_i) + \frac{1}{2}h_if^2_k(x_i) \\ &\approx 常数 + g_if_k(x_i) + \frac{1}{2}h_if^2_k(x_i) \end{aligned}$$

不难发现，该式子中$H_{k-1}(x_i)$是常数，因此第一部分$l(H_{t-1}(x_i))$也是一个常数，常数无法被最小化，因此我们可以将常数从该目标函数中剔除。经过泰勒展开，目标函数变为：

$$\begin{aligned} \tilde{O}_k &= \sum_{i=1}^M\left(g_if_k(x_i) + \frac{1}{2}h_if^2_k(x_i)\right) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2 \\ &= \sum_{i=1}^Mg_if_k(x_i) + \frac{1}{2}\sum_{i=1}^Mh_if^2_k(x_i) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2\end{aligned}$$

- **统一自变量**

现在目标函数的前两项分别代表所有样本的$g_if_k(x_i)$之和，以及所有样本的$h_if^2_k(x_i)$之和乘1/2。别忘记，我们选择的唯一的自变量是$w_j$，因此我们希望能够将$f_k$以某种方式转化为$w_j$。之前已经提到过多次，对任意位于叶子$j$上的样本$i$而言，数值上$f_k(x_i) = w_j$，我们可以尝试着从一个样本开始进行转化：

对于单一样本$i$，假设这个样本位于叶子$j$上，应该有：

$$g_if_k(x_i) = g_iw_j$$

对于一片叶子$j$，我们可以计算这片叶子上所有样本的$g_iw_j$之和：

$$\sum_{i \in j} g_iw_j $$

而一片叶子上所有样本的$w_j$都是一致的，因此一片叶子上的$g_iw_j$之和可以转变为：

$$\begin{aligned}\sum_{i \in j} g_iw_j &= g_1w_j \ + \ g_2w_j \ + \ ... \ + \ g_nw_j，其中1,2...n是叶子j上的样本 \\
&= w_j\sum_{i \in j} g_i\end{aligned}$$

假设现在一共有$T$片叶子，则整棵树上所有样本的$g_iw_j$之和为：

$$\sum_{j=1}^T \left( w_j\sum_{i \in j} g_i \right)$$

所以：

$$\sum_{i=1}^Mg_if_k(x_i) = \sum_{j=1}^T \left( w_j\sum_{i \in j} g_i \right)$$

同理，单一样本$i$的$h_if^2_k(x_i)$也可以以相同方式转化。对单一样本：

$$h_if^2_k(x_i) = h_iw^2_j$$

对一片叶子：

$$\begin{aligned}\sum_{i \in j}h_iw^2_j &= h_1w^2_j \ + \ h_2w^2_j \ + \ ... \ + \ h_nw^2_j，其中1,2...n是叶子j上的样本 \\ &= w^2_j\sum_{i \in j} h_i \end{aligned}$$

对整棵树：

$$\sum_{i=1}^Mh_if^2_k(x_i) = \sum_{j=1}^T \left( w^2_j\sum_{i \in j} h_i \right)$$

因此对整个目标函数有：

$$\begin{aligned} \tilde{O}_k &= \sum_{i=1}^Mg_if_k(x_i) + \frac{1}{2}\sum_{i=1}^Mh_if^2_k(x_i) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2 \\ 
&=\sum_{j=1}^T \left( w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j\sum_{i \in j} h_i \right) + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2\end{aligned}$$

不难发现，现在正则项可以与原来损失函数的部分合并了：

$$\begin{aligned}
 &= \sum_{j=1}^T \left( w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j\sum_{i \in j} h_i + \frac{1}{2}\lambda w_j^2 \right) + \gamma T \\ 
&= \sum_{j=1}^T \left( w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j(\sum_{i \in j} h_i + \lambda) \right) + \gamma T\end{aligned}$$

合并之后，整个目标函数变为两项，一项是所有叶子上的（损失+正则）之和，另一项是叶子总量。现在，我们可以开始求解最小目标函数以及对应的最优自变量$w_j$了。

## 3 求解XGBoost的目标函数

首先，令目标函数中的叶子总量最小是不可能的，过度降低叶子总量会大幅度伤害模型的学习能力，因此我们只能考虑令所有叶子上的（损失+正则）之和最小。

其次，当树建好之后，叶子与叶子之间是相互独立的，因此每片叶子上的（损失+正则）也是相互独立的。我们只要令每片叶子的（损失+正则）都最小，就可以保证全部叶子的（损失+正则）之和最小。故而，我们要令式子中标注为红色的部分最小：

$$\begin{aligned}
\tilde{O}_k 
&= \sum_{j=1}^T \left( \boldsymbol{\color{red}{w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j(\sum_{i \in j} h_i + \lambda)}} \right) + \gamma T\end{aligned}$$

- **叶子权重**$w_j$

将标注为红色的部分命名为$\mu_j$，表示叶子$j$上的损失+正则。则有：

$$\mu_j = w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j(\sum_{i \in j} h_i + \lambda)$$

**现在，对叶子$j$而言**，在$\mu_j$上对唯一自变量$w_j$求导，则有：

$$\begin{aligned}\frac{\partial{\mu_j}}{\partial w_j}
&= \frac{\partial{w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j(\sum_{i \in j} h_i + \lambda)}}{\partial w_j} \\ \\
&= \sum_{i \in j} g_i + w_j(\sum_{i \in j} h_i + \lambda)\end{aligned}$$

令一阶导数为0，则有：

$$\begin{aligned}
\sum_{i \in j} g_i + w_j(\sum_{i \in j} h_i + \lambda) &= 0 \\ \\
w_j(\sum_{i \in j} h_i + \lambda) &= -\sum_{i \in j} g_i \\ \\
w_j &= -\frac{\sum_{i \in j} g_i}{\sum_{i \in j} h_i + \lambda}\end{aligned}$$

你应该发现了，**对一片叶子来说，令目标函数最小的$w_j$就是我们之前提过的叶子权重，也就是XGBoost数学流程当中叶子上的输出值**。如果要令叶子的输出非常接近叶子权重公式，那应该如何拟合每个样本呢？

- **拟合值**

**对任意位于叶子$j$上的样本$i$来说**：

$$\mu_i = w_jg_i + \frac{1}{2}w^2_jh_i$$

将一片叶子上的$\mu_j$转变成$\mu_i$时，原则上需要将$\mu_j$中的每一项都转换为单个样本所对应的项，然而在转换正则项时则存在问题：与$\sum_{i \in j} g_i$这样可以直接指向单个样本的项不同，$\lambda$是针对与一片叶子设置的值，如果要将$\lambda$转变为针对单一样本的正则项，则需要知道当前叶子上一共有多少样本。然而，拟合发生在建树之前，因此在这一时间点不可能知道一片叶子上的样本总量，因此在xgboost的实际实现过程当中，**拟合每一片叶子时不涉及正则项**，只有在计算结构分数与叶子输出值时才使用正则项。

对$\mu_i$上唯一的自变量$w_j$求导，则有：

$$\begin{aligned}\frac{\partial{\mu_i}}{\partial w_j}
&= \frac{\partial{\left( w_jg_i + \frac{1}{2}w^2_jh_i \right)}}{\partial w_j} \\ \\
&= g_i + w_jh_i\end{aligned}$$

令一阶导数为0，则有：

$$\begin{aligned}
g_i + w_jh_i &= 0 \\ \\
w_jh_i &= - g_i \\ \\
w_j &= -\frac{g_i}{h_i} \end{aligned}$$

**对任意样本$i$而言，令目标函数最小的最优$w_j$就是我们的伪残差$r_i$，也就是XGBoost数学流程当中用于进行拟合的拟合值**。

- **结构分数**

现在，我们把令目标函数最小的最优$w_j$带回到$\mu_j$中，查看$\mu_j$如何变化：

$$\begin{aligned}
\mu_j &= w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j(\sum_{i \in j} h_i + \lambda) \\
&= -\frac{\sum_{i \in j} g_i}{\sum_{i \in j} h_i + \lambda} * \sum_{i \in j} g_i + \frac{1}{2}(-\frac{\sum_{i \in j} g_i}{\sum_{i \in j} h_i + \lambda})^2 * {\sum_{i \in j} h_i + \lambda}\\
&= -\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda} + \frac{1}{2}\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda} \\
&= - \frac{1}{2}\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda}
\end{aligned}$$

因此，目标函数（所有叶子上的损失）就可以变为：

$$\begin{aligned}
\tilde{O}_k 
&= \sum_{j=1}^T \left( \boldsymbol{\color{red}{w_j\sum_{i \in j} g_i + \frac{1}{2}w^2_j(\sum_{i \in j} h_i + \lambda)}} \right) + \gamma T \\ \\
&= \sum_{j=1}^T \left( -\frac{1}{2}\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda} \right) + \gamma T \end{aligned}
$$

因此，一片叶子上的目标函数就是：

$$ O_j = -\frac{1}{2}\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda} + \gamma$$

对任意一片叶子来说，目标函数可以衡量叶子的质量，其中$\gamma$是可以设定的超参数，$\frac{1}{2}$为常数，因此对任意叶子，我们希望标注为红色的部分越小越好：

$$ O_j = \frac{1}{2}\left( \boldsymbol{\color{red}{-\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda}}} \right)+ \gamma$$

故而，我们希望以下式子越大越好：

$$\frac{(\sum_{i \in j} g_i)^2}{\sum_{i \in j} h_i + \lambda}$$

这个式子，**正是XGBoost用于分枝时的指标“结构分数”（Structure Score）**。

- **结构分数的增益**

当分枝的时候，我们希望目标函数越小越好，因此在分枝过程中，父节点的目标函数是大于子节点的目标函数的，因此我们可以使用（父节点目标函数 - 子节点目标函数之和）来衡量分枝的质量，则有：

$$\begin{aligned}
Gain &= O_p - (O_l + O_r) \\ \\
&= -\frac{1}{2}\frac{(\sum_{i \in P} g_i)^2}{\sum_{i \in P} h_i + \lambda} + \gamma - (-\frac{1}{2}\frac{(\sum_{i \in L} g_i)^2}{\sum_{i \in L} h_i + \lambda} + \gamma  -\frac{1}{2}\frac{(\sum_{i \in R} g_i)^2}{\sum_{i \in R} h_i + \lambda} + \gamma) \\ \\
&= -\frac{1}{2}\frac{(\sum_{i \in P} g_i)^2}{\sum_{i \in P} h_i + \lambda} + \gamma + \frac{1}{2}\frac{(\sum_{i \in L} g_i)^2}{\sum_{i \in L} h_i + \lambda} - \gamma + \frac{1}{2}\frac{(\sum_{i \in R} g_i)^2}{\sum_{i \in R} h_i + \lambda} - \gamma \\ \\
&= \frac{1}{2}\left( \frac{(\sum_{i \in L} g_i)^2}{\sum_{i \in L} h_i + \lambda} + \frac{(\sum_{i \in R} g_i)^2}{\sum_{i \in R} h_i + \lambda} - \frac{(\sum_{i \in P} g_i)^2}{\sum_{i \in P} h_i + \lambda} \right) - \gamma \\ \\
&= \frac{1}{2} (Score_L + Score_R - Score_P) - \gamma
\end{aligned}$$

其中，$\gamma$是可以设定的超参数，$\frac{1}{2}$为常数，因此：

$$Gain = Score_L + Score_R - Score_P$$

**这就是我们在分枝时所使用的结构分数增益了**。

现在你发现了，XGBoost流程中所使用的全部新公式（包括独特的拟合值、独特的分枝指标、独特的输出值）都是通过令目标函数最小而求解出来的。因此，XGBoost整个流程就保证了目标函数一定是向着最小化方向进行迭代的，新生成的每片叶子上的输出值$w_j$都是会令目标函数最小化的输出值。现在，你可以回答以下的问题了：

- 1. 建树时拟合的$r_{ik} = -\frac{g_{ik}}{h_{ik}}$究竟是什么？拟合它有什么意义？

- 2. 结构分数和结构分数增益的公式是如何推导出来的？为什么这样建树可以提升模型的效果？

- 3. 为什么叶子节点的输出值$w_j$是$-\frac{(\sum_{i \in j} g_{ik})}{\sum_{i \in j} h_{ik} + \lambda}$？这样输出有什么意义？

- 4. 课程的第一部分说XGBoost拟合的也是残差，残差在哪里？

唯一余留问题4其实我们早已解答过，回去查看第二部分中的《2.2 弱评估器的分枝》，你会发现其实我们早已经求出残差了。当目标函数为$\frac{1}{2}MSE$，负梯度$-g_i$就等于残差，而$h_i = 1$，因此拟合项$-\frac{g_i}{h_i}$自然也是残差本身了。因此，XGBoost也是拟合负梯度的算法，并且在特定损失函数下，XGBoost也拟合残差。在了解这个推导过程之后，再返回复习XGBoost的整体数学流程，你会发现**数学真的让算法变得更简单，而不是更复杂**。如果你已经学到了这里，你已经跨过了机器学习原理中几乎最难的一部分。在接下来的课程中，所有数学推导对你来说都不再是阻碍了。